# Setup

In [ ]:
import sys
import numpy as np
import pandas as pd
import datetime
from math import log as log
import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
import joblib

In [ ]:
RANDOM_SEED = 11

In [ ]:
sys.path.append('/Users/rus/Desktop/Credit-Scoring/modules')

# вывести информацию о полях и размерности датасета
from DataAnalysis import primary_info_about_data
# построить боксплоты для столбцов датафрейма
from DataAnalysis import boxplot_create

# сгенерировать признаки для датафрейма, вернуть полный датафрейм с новыми признаками
from FeatureEngineering import feature_eng

# F-test
from FTest import f_num, f_cat

# Построить график roc auc, Построить график матрицы корреляции
from Metrics import roc_auc_create, confusion_matrix_create
# Рассчитать метрики качества классификатора
from Metrics import model_metrics

# Протестировать модели с базовыми параметрами, вывести метрики для каждой
from Models import test_base_models
# Подбор гиперпараметров и сохранение модели с лучшим показателем
from Models import tune_model
# Формирование файла с вероятностью дефолта
from Models import sub_create

# Данные

In [ ]:
# путь к файлам
path = 'data'

In [ ]:
train = pd.read_csv(path +'/train.csv')
test = pd.read_csv(path +'/test.csv')
sample = pd.read_csv(path +'/sample_submission.csv')

### Рассмотрим данные

#### train

In [ ]:
primary_info_about_data(train)
train.head()

#### test

In [ ]:
primary_info_about_data(test)
test.head()

#### sample

In [ ]:
primary_info_about_data(sample)
sample.head()

#### Распределение целевого признака

In [ ]:
plot = sns.countplot(x=train['default'])

### Объединим train и test

In [ ]:
train['sample'] = 1
test['sample'] = 0
test['default'] = -1  # временное значение
data = train.append(test, sort=False).reset_index(drop=True)

In [ ]:
primary_info_about_data(data)

<h2>Описания полей</h2>
<ul>
<li><p><code>client_id</code> - идентификатор клиента</p></li>
<li><p><code>education</code> - уровень образования</p></li>
<li><p><code>sex</code> - пол заемщика</p></li>
<li><p><code>age</code> - возраст заемщика</p></li>
<li><p><code>car</code> - флаг наличия автомобиля</p></li>
<li><p><code>car_type</code> - флаг автомобиля иномарки</p></li>
<li><p><code>decline_app_cnt</code> - количество отказанных прошлых заявок</p></li>
<li><p><code>good_work</code> - флаг наличия “хорошей” работы</p></li>
<li><p><code>bki_request_cnt</code> - количество запросов в БКИ</p></li>
<li><p><code>home_address</code> - категоризатор домашнего адреса</p></li>
<li><p><code>work_address</code> -  категоризатор рабочего адреса</p></li>
<li><p><code>income</code> - доход заемщика</p></li>
<li><p><code>foreign_passport</code> - наличие загранпаспорта</p></li>
<li><p><code>sna</code> - связь заемщика с клиентами банка</p></li>
<li><p><code>first_time</code> - давность наличия информации о заемщике</p></li>
<li><p><code>score_bki</code> - скоринговый балл по данным из БКИ</p></li>
<li><p><code>region_rating</code> - рейтинг региона</p></li>
<li><p><code>app_date</code> - дата подачи заявки</p></li>
<li><p><code>default</code> - флаг дефолта по кредиту </p></li>
</ul></div>

# Разведочный анализ (EDA)

In [ ]:
# разделяю по типам признаков для предобработки
num_cols = ['age','decline_app_cnt','score_bki','income','bki_request_cnt','region_rating'] 
cat_cols = ['education','work_address','home_address','sna','first_time'] 
bin_cols = ['sex','car','car_type','good_work','foreign_passport'] 

## Числовые признаки

In [ ]:
# рассмотрим числовые значения
fig, axes = plt.subplots(2, 3, figsize=(25,15))

for i,col in enumerate(num_cols):
    sns.histplot(data[col], kde=False, ax=axes.flat[i], color="r")

- чаще заемщиками являются клиенты 25-35
- большинство не имеет отказных заявок, но есть небольшое число клиентов с высоким количеством отказов, отказы смещены влево

Определить характеристику клиентов с высокой отказностью

- оценка bki распределена нормально
- доход смещен влево, есть выбросы с крайне высоким уровнем дохода

Есть ли те, кто не имеет "хорошую" работу, но имеет доход выше среднего?

- половина клиентов имеет 0 или 1 запрос в БКИ, данные смещены влево
- крайне мало клиентов из регионов с рейтиного 30 и ниже

### Логорифмируем смещенные признаки для нормального распределения

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(10,7))
for i,col in enumerate(['decline_app_cnt', 'bki_request_cnt', 'income']):
    data[col] = np.log(data[col] + 1)
    sns.histplot(data[col][data[col] > 0].dropna(), ax=axes.flat[i],kde = False, color="r")    

### Построим боксплоты для числовых признаков

In [ ]:
for col in num_cols:
    boxplot_create(data, col)

- Молодые более склонны к совершению дефолта
- Дефолт совершают люди, которые имеют более высокое значение скоринговой оценки
- Платежеспособные люди живут, как правило, в регионах с более высоким рейтингом
- У совершающих дефолт доход чуть ниже
- Количество запросов в БКИ, обычно, выше у совершающих дефолт

## Категориальные признаки

### Образование

In [ ]:
data.education.value_counts().plot(kind="bar",figsize=(8,6),color="r")
print("Missing education values:\n",data.education.isna().sum())

In [ ]:
data.education = data.education.fillna("SCH") # заменим пустые значения на самое популярное

Смотрим зависимость дохода от уровня образования

In [ ]:
plt.figure(figsize=(15, 8))
sns.boxplot(x="education", y="income", data=data, showfliers=False)

Люди с более высоким образованием имеют, как правило, более высокий доход

## Построим корреляционную матрицу

In [ ]:
plt.rcParams['figure.figsize'] = (15,10)
sns.heatmap(data[data['sample'] == 1].corr(), vmin=0, vmax=1, annot = True)

- Признаки "домашний" и "рабочий" адресы имеют высокую зависимость - большинство выбирает работу рядом с домом
- bki score, количество отказанных прошлых заявок и sna имеют наибольшую корреляцию с целевым признаком

# Feature engineering

In [ ]:
# сгенерируем признаки
df = feature_eng(data)

## Новые признаки:
- app_date - Сколько дней прошло с 1-го запроса
- home_work - Дом далеко от работы
- bki/decline - Количество отказных заявок/Количество запросов
- age_category - Категории возрастов
- mean_income_age_cat - средний доход на категорию возраста (временный признак)
- inc_large_mean - Доход больше среднего по категории возраста
- normalized_income_minus_mean - Нормализованный доход по категории возраста
- mean_requests_age - среднее число обращений в БКИ на категорию возраста (временный признак)
- bki_request_cnt - Больше среднего обращений в БКИ по категории возраста
- mean_income_region - средний доход по региону (временный признак)
- inc_large_mean_region - Доход больше среднего по региону
- active_no_decline - Без отказов от банков, но имеет обращения в БКИ
- no_decline_request - Не имеет запросов в БКИ и не имеет отказов
- no_good_job_and_good_income - Имеет доход выше среднего, но не имеет пометку "хорошая" работа (бизнес?)
- income_per_score_bki - Доход/Скоринг оценку

## Кодирование бинарных и категориальных признаков признаков

In [ ]:
cat_cols = ['education','work_address','home_address','sna','first_time', 'age_category'] 

In [ ]:
# перекодируем текстовые значения в бинарных признаках на числовые
labels = {}
label_encoder = LabelEncoder()
for col in bin_cols+cat_cols:
    # перекодировка
    df[col] = label_encoder.fit_transform(df[col])
    # сохраняем значения перекодировки в словаре
    labels[col] = dict(enumerate(label_encoder.classes_))

In [ ]:
# Словарь категориальных и бинарных признаков
print(labels)

In [ ]:
# обновляем списки признаков по типу
num_cols = ['age','decline_app_cnt','score_bki','income','bki_request_cnt','app_date', 'region_rating',
            'bki/decline', 'normalized_income_minus_mean',
           'income_per_score_bki'] # numerical
cat_cols = ['education','work_address','home_address','sna','first_time', 'age_category'] # categorical
bin_cols = ['sex','car','car_type','good_work','foreign_passport','home_work', 'inc_large_mean',
           'request_large_mean', 'inc_large_mean_region', 'active_no_decline', 'no_decline_request',
           'no_good_job_and_good_income', ] # binary

## Обработка выбросов

In [ ]:
# построим боксплоты
for col in num_cols:
    boxplot_create(df, col)

In [ ]:
# приведем к положительному значению
df['score_bki'] = abs(df['score_bki'])

In [ ]:
# логарифмируем числовые признаки
for i in num_cols:
     df[i] = np.log(df[i])
     df[i].replace([np.inf, -np.inf], 0, inplace=True)

## F-test

оцениваю разделяющую способность признака к целевой переменной

In [ ]:
# тренировочная выборка
data_temp = df.loc[df['sample'] == 1] 

### Числовые признаки

In [ ]:
f_num(data_temp, num_cols)

### Категориальные и бинарные признаки

In [ ]:
f_cat(data_temp, bin_cols+cat_cols)

## Собираю готовый dataframe

In [ ]:
df = pd.get_dummies(df, prefix=cat_cols, columns=cat_cols) # dummy-переменные

# Обучим модель

## Baseline

In [ ]:
# разделяю на тренировочную и тестовую выборки
train_df = df.query('sample == 1').drop(['sample','client_id'], axis=1)
test_df = df.query('sample == 0').drop(['sample','client_id'], axis=1)

In [ ]:
X = train_df.drop(['default'], axis=1).values
y = train_df['default'].values # целевой признак

In [ ]:
# разделяю выборку на тренировочную и валидационную
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.20, random_state=RANDOM_SEED)

In [ ]:
lr = LogisticRegression(max_iter = 1000)

In [ ]:
lr.fit(X_train, y_train)
y_pred = lr.predict(X_valid)
probs = lr.predict_proba(X_valid)

### ROC AUC

In [ ]:
# построим график roc auc
roc_auc_create(probs, y_valid)

In [ ]:
# рассчитаем метрики для модели
model_metrics(y_valid,y_pred)

Модель имеет низкие показатели

### Confusion matrix

In [ ]:
# построим матрицу корреляций
confusion_matrix_create(y_valid, y_pred)

Модель предсказывает, что практически все не дефолтные

## Разделим данные

In [ ]:
data = df.copy()

In [ ]:
# размеченные данные
train_data = data.query('sample == 1').drop(['sample','client_id'], axis=1)
# данные для предикта
test_data = data.query('sample == 0').drop(['sample','client_id'], axis=1)

X_train = train_data.drop(['default'], axis=1)
y_train = train_data.default.values
X_test = test_data.drop(['default'], axis=1)

In [ ]:
X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(X_train, y_train, test_size=0.2, shuffle=True, random_state=RANDOM_SEED)

## Протестируем алгоритмы с дефолтными параметрами

In [ ]:
test_base_models(X_train_1, y_train_1, X_test_1, y_test_1, RANDOM_SEED)

#### SVC

Всех обозначил не дефолтными

- precision_score: 0.0
- recall_score: 0.0
- f1_score: 0.0

#### KNeighborsClassifier
- precision_score: 0.27176220806794055
- recall_score: 0.06863270777479893
- f1_score: 0.10958904109589042

#### MLPClassifier
- precision_score: 0.32840236686390534
- recall_score: 0.05951742627345844
- f1_score: 0.10077167498865182

#### XGBClassifier
- precision_score: 0.3561643835616438
- recall_score: 0.0418230563002681
- f1_score: 0.07485604606525913

## Подберем параметры и сделаем предсказание

### XGBClassifier

In [ ]:
# параметры для перебора
params = {
        'learning_rate' : [0.05, 0.1, 0.2,],
        'n_estimators' : [150, 200, 250],
        'min_child_weight': [4, 5, 6],
        'gamma': [0.3, 0.5, 0.8],
        'subsample': [0.9, 1.0, 1.2],
        'colsample_bytree': [0.5, 0.6, 0.7],
        'max_depth': [4, 5, 6]
        }

In [ ]:
# инициализируем модель
model = XGBClassifier()
name = 'xgb'

In [ ]:
# подберем параметры и сохраним лучшую модель
tune_model(model, params, X_train, y_train, RANDOM_SEED, name)

In [ ]:
# сделаем предсказание
sub_create('save_models/xgb_model.pkl', X_test, test, name)

Sub_result: 0.74268

### KNeighborsClassifier

In [ ]:
# параметры для перебора
params = {    
        'n_neighbors': list(range(2, 11, 2)),
        'leaf_size': list(range(20, 41, 3)),
        'p': [1,2],
        'weights': ['uniform', 'distance'],
        'metric': ['minkowski', 'chebyshev'],
        }

In [ ]:
# инициализируем модель
model = KNeighborsClassifier()
name = 'knn'

In [ ]:
tune_model(model, params, X_train, y_train, RANDOM_SEED, name)

In [ ]:
# сделаем предсказание
sub_create('save_models/knn_model.pkl', X_test, test, name)

Sub_result: 0.62991

### MLPClassifier

In [ ]:
# параметры для перебора
params = {    
        'hidden_layer_sizes': [(50,50,50), (50,100,50), (100,)],
        'activation': ['tanh', 'relu'],
        'solver': ['sgd', 'adam'],
        'alpha': [0.0001, 0.05],
        'learning_rate': ['constant','adaptive'],
        }

In [ ]:
# инициализируем модель
model = MLPClassifier(random_state=RANDOM_SEED)
name = 'mlp'

In [ ]:
# подберем параметры и сохраним лучшую модель
tune_model(model, params, X_train, y_train, RANDOM_SEED, name)

In [ ]:
# сделаем предсказание
sub_create('save_models/mlp_model.pkl', X_test, test, name)

Sub_result: 0.73829

**Лучший результат на тестовой выборке показал xgb классификатор**